### Libraries

In [1]:
! pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.2 MB/s 
     |████████████████████████████████| 4.4 MB 34.0 MB/s 
     |████████████████████████████████| 175 kB 52.7 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
     |████████████████████████████████| 101 kB 8.3 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=a84eab2431717338230f05079d7816af1e598b9276336090c3fd5b1872cb046a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! mkdir files
! cp "drive/MyDrive/Università/5/Stage/tweets-analysis/TweetsSentiment.py" .
! cp "drive/MyDrive/Università/5/Stage/tweets-analysis/TweetsUtils.py" .
! cp "drive/MyDrive/Università/5/Stage/tweets-analysis/files/Italian-it-NRC-VAD-Lexicon_scaled (version 1).csv" files
! cp "drive/MyDrive/Università/5/Stage/tweets-analysis/files/codici_emoji.xlsx" files

In [6]:
from TweetsUtils import *
import TweetsSentiment

In [7]:
base_path = '/content/drive/MyDrive/Università/5/Stage/tweets-analysis/'

### import files

In [8]:
lang = 'it'

years = [2018,2019,2021,2022]

# dates = {2018: ('2018-04-17 00:00:00', '2018-04-22 23:59:59'),
#          2019: ('2019-04-08 00:00:00', '2019-04-14 23:59:59'), 
#          2021: ('2021-09-04 00:00:00', '2021-09-10 23:59:59'), 
#          2022: ('2022-06-06 00:00:00', '2022-06-12 23:59:59')}

In [9]:
def read_tweets(years, lang):
    tweets, users, places = [], [], []
    for year in years:
        tmp_tweets, tmp_users, tmp_places = read_tweets_files(year, lang)
        tweets += tmp_tweets
        users += tmp_users
        places += tmp_places
    return tweets, users, places



def read_tweets_files(year, lang):
    if lang == 'en':
        lang = '_en'
    else:
        lang = ''
    
    tmp_path = base_path + 'tweets/mdw' + str(year)[-2:] + lang + '_tweets/'
    tweets_filename = tmp_path + 'mdw' + str(year)[-2:] + lang + '_tweets_contents.json'
    users_filename = tmp_path + 'mdw' + str(year)[-2:] + lang + '_tweets_users.json'
    places_filename = tmp_path + 'mdw' + str(year)[-2:] + lang + '_tweets_places.json'
    
    tmp_tweets = read_file(tweets_filename)
    tmp_users = read_file(users_filename)
    tmp_places = read_file(places_filename)
    
    return tmp_tweets, tmp_users, tmp_places

In [10]:
_, users, places = read_tweets(years, lang)
tweets = read_file(base_path+'tweets/total_tweets.json')

print('TOT:', len(tweets), 'tweets')
print('TOT:', len(users), 'users')
print('TOT:', len(places), 'places')

TOT: 71061 tweets
TOT: 30182 users
TOT: 286 places


In [11]:
# ignoro i tweet riguardo il salone del mobile di torino
tweets = [tw for tw in tweets if not ('torino' in tw['text'].lower() and 'milano' not in tw['text'].lower())]
len(tweets)

70788

In [12]:
# testi unici
unique_texts = select_fields(tweets, ['text'], as_list=True, unique=True)
len(unique_texts)

45373

### only ner

In [ ]:
import stanza
stanza.download(lang)

2022-07-04 19:49:22 INFO: Downloading default packages for language: it (Italian)...


2022-07-04 19:49:40 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang, processors='tokenize,ner')

2022-07-04 19:49:40 WARNING: Language it package default expects mwt, which has been added
2022-07-04 19:49:40 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| ner       | fbk      |

2022-07-04 19:49:40 INFO: Use device: cpu
2022-07-04 19:49:40 INFO: Loading: tokenize
2022-07-04 19:49:40 INFO: Loading: mwt
2022-07-04 19:49:40 INFO: Loading: ner
2022-07-04 19:49:41 INFO: Done loading processors!


In [ ]:
def get_entities(text):
    # remove URLs
    regex = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'
    text = re.sub(regex, '', text).strip()

    # remove html chars
    for sequence in ['&gt', '&lt', '&amp', ';quot', '...', '…']:
        text = text.replace(sequence, ' ')
    text = text.replace('\n', '. ')
    text = text.encode("utf-8", "ignore").decode()
    text = ' '.join(text.split())

    # tokenization, lemmatization, ner
    processed = nlp(text)

    # reformat entities
    entities = [e.to_dict() for e in processed.entities]
    entities = select_fields(entities, ['text', 'type'])
    
    return entities

In [ ]:
import os
files = os.listdir(base_path+'tmp/')
tmp = []
for f in files:
  tmp += list(read_file(base_path+'tmp/'+f).keys())
len(tmp)

43446

In [ ]:
batch_size = 2000

i = 0
while i < len(unique_texts):
  d = {text: get_entities(text) for text in unique_texts[i:(i+batch_size)] if text not in tmp}
  save_file(d, base_path+'tmp/tmp3_'+str(i)+'.json')
  print(i)
  i += batch_size

In [ ]:
# leggere tutti i file, unirli, aggiungere ai tweets le entità, salvare tutto in un total_tweets_with_entities.json

In [13]:
from collections import ChainMap
import os

files = os.listdir(base_path+'tmp/')
tmp = [read_file(base_path+'tmp/'+f) for f in files]
print(len(tmp))

d = dict(ChainMap(*tmp))
print(len(d))

38
43446


In [14]:
for i in range(len(tweets)):
  tw = tweets[i]
  if tw['text'] in d:
    tw['entities'] = d[tw['text']]
  else:
    tw['entities'] = []

In [15]:
save_file(tweets, base_path+'tweets/total_tweets_entities.json')

### sentiment analysis

In [ ]:
unique_texts = select_fields(tweets, ['text'], as_list=True, unique=True)
len(unique_texts)

43446

In [ ]:
ts = TweetsSentiment.TweetsSentiment('italian')
ts.setup_stanza()

2022-06-15 12:12:42 INFO: Downloading default packages for language: it (Italian)...
2022-06-15 12:12:43 INFO: File exists: /root/stanza_resources/it/default.zip
2022-06-15 12:12:49 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-06-15 12:12:50 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| pos       | combined |
| lemma     | combined |
| ner       | fbk      |

2022-06-15 12:12:50 INFO: Use device: cpu
2022-06-15 12:12:50 INFO: Loading: tokenize
2022-06-15 12:12:50 INFO: Loading: mwt
2022-06-15 12:12:50 INFO: Loading: pos
2022-06-15 12:12:50 INFO: Loading: lemma
2022-06-15 12:12:50 INFO: Loading: ner
2022-06-15 12:12:51 INFO: Done loading processors!


In [ ]:
d = {text: ts.process_tweet(text) for text in unique_texts}

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
for i in range(len(tweets)):
    tw = tweets[i]
    if tw['text'] in d:
        tweets[i] = {**tw, **d[tw['text']]}

In [ ]:
save_file(tweets, dest_filename)